# Gamma Scalping Strategy Analysis
## Interactive visualization and comprehensive analysis using Plotly

This notebook provides a complete analysis of the gamma scalping strategy including:
- Strategy performance across different market periods
- Greeks evolution and hedging dynamics
- P&L attribution and risk analysis
- Interactive strategy comparison dashboard

In [12]:
# Setup and imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Set plotly theme
pio.templates.default = "plotly_white"

# Add parent directory to path
sys.path.append(str(Path().absolute().parent))

from data.data_loader import DataLoader
from simulation.gamma_scalping_simulator import GammaScalpingSimulator
from strategies.gamma_scalping import run_portfolio_backtest
from backtest.backtest_engine import StrategyConfig
from models.options_pricing import delta, gamma, theta, vega, bs_price, implied_volatility

print("📊 Gamma Scalping Analysis Notebook")
print("✅ Libraries loaded successfully!")

# Change to project root if in notebooks directory
import os
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
    print(f"📁 Changed directory to: {os.getcwd()}")

📊 Gamma Scalping Analysis Notebook
✅ Libraries loaded successfully!


## 1. Data Loading and Overview
Load market data and get an overview of available options

In [13]:
# Load a comprehensive dataset
print("📥 Loading market data...")
start_date = pd.Timestamp('2025-01-01')
end_date = pd.Timestamp('2025-03-01')

# Initialize data loader
loader = DataLoader()
spot_data, options_dict = loader.create_dataset_dict(
    spot_symbol='BTCUSDT',
    underlying='BTC',
    start_date=start_date,
    end_date=end_date,
    option_filters={'min_volume': 0}  # Include all options
)

total_option_rows = sum(len(df) for df in options_dict.values()) if options_dict else 0
print(f"📈 Loaded {len(spot_data):,} spot prices")
print(f"📊 Loaded {total_option_rows:,} option data rows across {len(options_dict)} contracts")
print(f"📅 Date range: {spot_data['timestamp'].min().date()} to {spot_data['timestamp'].max().date()}")
print(f"💰 Spot price range: ${spot_data['close'].min():,.0f} - ${spot_data['close'].max():,.0f}")

if options_dict:
    print(f"🎯 Unique option contracts: {len(options_dict):,}")

# Create spot price chart
fig_spot = px.line(
    spot_data, 
    x='timestamp', 
    y='close',
    title='BTC Spot Price Evolution',
    labels={'close': 'Price (USD)', 'timestamp': 'Date'}
)
fig_spot.update_layout(height=400)
fig_spot.show()

📥 Loading market data...
📈 Loaded 1,417 spot prices
📊 Loaded 88,171 option data rows across 1595 contracts
📅 Date range: 2025-01-01 to 2025-03-01
💰 Spot price range: $78,976 - $108,320
🎯 Unique option contracts: 1,595
📈 Loaded 1,417 spot prices
📊 Loaded 88,171 option data rows across 1595 contracts
📅 Date range: 2025-01-01 to 2025-03-01
💰 Spot price range: $78,976 - $108,320
🎯 Unique option contracts: 1,595


## 2. Strategy Performance Analysis
Run gamma scalping backtests and analyze performance

In [14]:
# Define strategy configurations
strategies = [
    StrategyConfig(
        name="Conservative (15%)",
        hedge_threshold=0.15,
        option_selection='atm',
        position_size=1.0
    ),
    StrategyConfig(
        name="Moderate (10%)",
        hedge_threshold=0.10,
        option_selection='atm',
        position_size=1.0
    ),
    StrategyConfig(
        name="Aggressive (5%)",
        hedge_threshold=0.05,
        option_selection='atm',
        position_size=1.0
    ),
    StrategyConfig(
        name="Very Aggressive (2.5%)",
        hedge_threshold=0.025,
        option_selection='atm',
        position_size=1.0
    )
]

print("🚀 Running strategy backtests...")
print("This may take a few minutes...")

# Run backtests with raw data return for detailed analysis
payload = run_portfolio_backtest(
    strategy_configs=strategies,
    spot_data=spot_data,
    options_dict=options_dict,
    max_options=30,
    target_dte=30,
    moneyness_filter='ATM',
    select='filtered',
    workers=1,
    return_raw=True,
    aggregate_timeseries=False,
)

# Extract results
results = payload['summary_df']
raw_results_map = payload.get('raw_results')  # dict[strategy->list[GammaScalpingResult]] or flat list

print(f"✅ Backtests completed! Analyzed {len(results)} strategy-option combinations")

🚀 Running strategy backtests...
This may take a few minutes...
GAMMA SCALPING BACKTEST
Period: 2025-01-01 to 2025-03-01
Target DTE: 30 days
Moneyness filter: ATM
Selection: filtered by DTE/moneyness/liquidity

Loaded spot rows: 1,417 | options rows total: 88,171

Analyzing option chain...
Selected 30 options for testing
Strike range: 86000 - 104000
DTE range: 20 - 30 days

--- Testing strategy: Conservative (15%) ---
Selected 30 options for testing
Strike range: 86000 - 104000
DTE range: 20 - 30 days

--- Testing strategy: Conservative (15%) ---


Options: 100%|██████████| 30/30 [00:04<00:00,  6.85it/s]



Results for Conservative (15%):
  Total P&L: $54,829.34
  Average P&L per option: $1,827.64
  Win rate: 56.7%
  Options traded: 30

--- Testing strategy: Moderate (10%) ---


Options: 100%|██████████| 30/30 [00:04<00:00,  6.87it/s]



Results for Moderate (10%):
  Total P&L: $53,244.20
  Average P&L per option: $1,774.81
  Win rate: 56.7%
  Options traded: 30

--- Testing strategy: Aggressive (5%) ---


Options: 100%|██████████| 30/30 [00:04<00:00,  6.81it/s]



Results for Aggressive (5%):
  Total P&L: $51,672.50
  Average P&L per option: $1,722.42
  Win rate: 56.7%
  Options traded: 30

--- Testing strategy: Very Aggressive (2.5%) ---


Options: 100%|██████████| 30/30 [00:04<00:00,  6.68it/s]


Results for Very Aggressive (2.5%):
  Total P&L: $51,099.79
  Average P&L per option: $1,703.33
  Win rate: 56.7%
  Options traded: 30

SUMMARY BY STRATEGY
                                       total_pnl                   option_pnl  \
                                             sum     mean      std        sum   
strategy               hedge_threshold                                          
Aggressive (5%)        0.050            51672.50  1722.42  3605.11   49743.21   
Conservative (15%)     0.150            54829.34  1827.64  3667.33   49743.21   
Moderate (10%)         0.100            53244.20  1774.81  3655.48   49743.21   
Very Aggressive (2.5%) 0.025            51099.79  1703.33  3585.50   49743.21   

                                       hedge_pnl commission_cost  \
                                             sum             sum   
strategy               hedge_threshold                             
Aggressive (5%)        0.050            34649.41        10906.71   
Con

In [15]:
# Create performance dashboard
if not results.empty:
    # Aggregate results by strategy
    strategy_summary = results.groupby('strategy').agg({
        'total_pnl': 'sum',
        'option_pnl': 'sum', 
        'hedge_pnl': 'sum',
        'commission_cost': 'sum',
        'slippage_cost': 'sum',
        'num_hedges': 'mean',
        'gamma_pnl': 'sum',
        'theta_cost': 'sum'
    }).reset_index()
    
    strategy_summary['win_rate'] = results.groupby('strategy')['total_pnl'].apply(lambda x: (x > 0).mean()).values
    strategy_summary['num_options'] = results.groupby('strategy').size().values
    
    # Create subplot dashboard
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Total P&L by Strategy',
            'Win Rate vs Average Hedges',
            'P&L Attribution',
            'Transaction Costs',
            'Gamma vs Theta Effects',
            'Strategy Comparison'
        ],
        specs=[[{'type': 'bar'}, {'type': 'bar'}],
               [{'type': 'bar'}, {'type': 'bar'}]]
    )
    
    # 1. Total P&L by Strategy
    colors = ['green' if x > 0 else 'red' for x in strategy_summary['total_pnl']]
    fig.add_trace(
        go.Bar(
            x=strategy_summary['strategy'],
            y=strategy_summary['total_pnl'],
            name='Total P&L',
            marker_color=colors,
            showlegend=False
        ),
        row=1, col=1
    )
    
    # 3. P&L Attribution
    fig.add_trace(
        go.Bar(
            x=strategy_summary['strategy'],
            y=strategy_summary['option_pnl'],
            name='Option P&L',
            marker_color='lightblue'
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Bar(
            x=strategy_summary['strategy'],
            y=strategy_summary['hedge_pnl'],
            name='Hedge P&L',
            marker_color='lightcoral'
        ),
        row=1, col=2
    )
    
    # 4. Transaction Costs
    total_costs = strategy_summary['commission_cost'] + strategy_summary['slippage_cost']
    fig.add_trace(
        go.Bar(
            x=strategy_summary['strategy'],
            y=total_costs,
            name='Transaction Costs',
            marker_color='orange',
            showlegend=False
        ),
        row=2, col=1
    )
    
    # 6. Strategy Ranking
    strategy_summary_sorted = strategy_summary.sort_values('total_pnl', ascending=True)
    fig.add_trace(
        go.Bar(
            y=strategy_summary_sorted['strategy'],
            x=strategy_summary_sorted['total_pnl'],
            orientation='h',
            name='Strategy Ranking',
            marker_color=['green' if x > 0 else 'red' for x in strategy_summary_sorted['total_pnl']],
            showlegend=False
        ),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text="Gamma Scalping Strategy Performance Dashboard",
        title_x=0.5,
        showlegend=True
    )
    
    # Update axis labels
    fig.update_xaxes(title_text="Strategy", row=1, col=1)
    fig.update_yaxes(title_text="P&L ($)", row=1, col=1)
    
    fig.update_xaxes(title_text="Average Hedges", row=1, col=2)
    fig.update_yaxes(title_text="Win Rate (%)", row=1, col=2)
    
    fig.update_xaxes(title_text="Strategy", row=1, col=3)
    fig.update_yaxes(title_text="P&L ($)", row=1, col=3)
    
    fig.update_xaxes(title_text="Strategy", row=2, col=1)
    fig.update_yaxes(title_text="Cost ($)", row=2, col=1)
    
    fig.update_xaxes(title_text="Gamma P&L ($)", row=2, col=2)
    fig.update_yaxes(title_text="Theta Cost ($)", row=2, col=2)
    
    fig.update_xaxes(title_text="Total P&L ($)", row=2, col=3)
    fig.update_yaxes(title_text="Strategy", row=2, col=3)
    
    fig.show()
    
    # Print summary statistics
    print("\n📊 Strategy Performance Summary:")
    print("=" * 50)
    for _, row in strategy_summary.iterrows():
        print(f"\n{row['strategy']}:")
        print(f"  💰 Total P&L: ${row['total_pnl']:,.2f}")
        print(f"  🎯 Win Rate: {row['win_rate']:.1%}")
        print(f"  📈 Options Traded: {row['num_options']}")
        print(f"  🔄 Avg Hedges: {row['num_hedges']:.1f}")

else:
    print("❌ No backtest results available")


📊 Strategy Performance Summary:

Aggressive (5%):
  💰 Total P&L: $51,672.50
  🎯 Win Rate: 56.7%
  📈 Options Traded: 30
  🔄 Avg Hedges: 17.4

Conservative (15%):
  💰 Total P&L: $54,829.34
  🎯 Win Rate: 56.7%
  📈 Options Traded: 30
  🔄 Avg Hedges: 12.4

Moderate (10%):
  💰 Total P&L: $53,244.20
  🎯 Win Rate: 56.7%
  📈 Options Traded: 30
  🔄 Avg Hedges: 14.4

Very Aggressive (2.5%):
  💰 Total P&L: $51,099.79
  🎯 Win Rate: 56.7%
  📈 Options Traded: 30
  🔄 Avg Hedges: 21.7


## 3. Greeks Evolution Analysis
Analyze how option Greeks evolve over time and their impact on P&L

In [16]:
def analyze_option_greeks(spot_data, strike=30000, expiry_days=30, iv=0.8):
    """Analyze Greeks evolution for a specific option"""
    
    expiry = spot_data['timestamp'].min() + pd.Timedelta(days=expiry_days)
    risk_free_rate = 0.01
    
    greeks_data = []
    
    for _, row in spot_data.iterrows():
        spot = row['close']
        timestamp = row['timestamp']
        
        # Calculate time to expiry
        tte = max(0, (expiry - timestamp).total_seconds() / (365 * 24 * 3600))
        
        if tte > 0:
            # Calculate Greeks for call option
            call_delta = delta(spot, strike, tte, risk_free_rate, iv, 'call')
            call_gamma = gamma(spot, strike, tte, risk_free_rate, iv)
            call_theta = theta(spot, strike, tte, risk_free_rate, iv, 'call')
            call_vega = vega(spot, strike, tte, risk_free_rate, iv)
            call_price = bs_price(spot, strike, tte, risk_free_rate, iv, 'call')
            
            greeks_data.append({
                'timestamp': timestamp,
                'spot': spot,
                'strike': strike,
                'tte': tte * 365,  # Days
                'delta': call_delta,
                'gamma': call_gamma,
                'theta': call_theta / 365,  # Daily
                'vega': call_vega / 100,  # Per 1% IV
                'option_price': call_price,
                'moneyness': spot / strike,
                'intrinsic_value': max(0, spot - strike)
            })
    
    return pd.DataFrame(greeks_data)

# Analyze Greeks for a representative option
print("📈 Analyzing Greeks evolution...")
initial_spot = spot_data['close'].iloc[0]
atm_strike = round(initial_spot / 1000) * 1000  # Round to nearest 1000

greeks_df = analyze_option_greeks(
    spot_data=spot_data.iloc[:1000],  # First ~40 days of data
    strike=atm_strike,
    expiry_days=30,
    iv=0.8
)

print(f"🎯 Analyzing ATM option with strike ${atm_strike:,}")
print(f"📊 Generated {len(greeks_df)} data points")

📈 Analyzing Greeks evolution...
🎯 Analyzing ATM option with strike $94,000
📊 Generated 720 data points


In [17]:
# Create comprehensive Greeks visualization
fig_greeks = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Spot Price and Option Value',
        'Delta Evolution',
        'Gamma (Sensitivity)',
        'Theta Decay',
    ],
    specs=[[{'secondary_y': True}, {'type': 'scatter'}],
           [{'type': 'scatter'}, {'type': 'scatter'}]]
)

# 1. Spot Price and Option Value
fig_greeks.add_trace(
    go.Scatter(
        x=greeks_df['timestamp'],
        y=greeks_df['spot'],
        name='Spot Price',
        line=dict(color='black', width=2)
    ),
    row=1, col=1, secondary_y=False
)

fig_greeks.add_trace(
    go.Scatter(
        x=greeks_df['timestamp'],
        y=greeks_df['option_price'],
        name='Option Price',
        line=dict(color='blue', width=2)
    ),
    row=1, col=1, secondary_y=True
)

# 2. Delta Evolution
fig_greeks.add_trace(
    go.Scatter(
        x=greeks_df['timestamp'],
        y=greeks_df['delta'],
        name='Delta',
        line=dict(color='green', width=2)
    ),
    row=1, col=2
)
fig_greeks.add_hline(y=0.5, line_dash="dash", line_color="red", 
                    annotation_text="ATM Delta", row=1, col=2)

# 3. Gamma
fig_greeks.add_trace(
    go.Scatter(
        x=greeks_df['timestamp'],
        y=greeks_df['gamma'] * 10000,  # Scale for readability
        name='Gamma (×10k)',
        line=dict(color='orange', width=2),
        fill='tonexty'
    ),
    row=2, col=1
)

# 4. Theta
fig_greeks.add_trace(
    go.Scatter(
        x=greeks_df['timestamp'],
        y=greeks_df['theta'],
        name='Daily Theta',
        line=dict(color='red', width=2),
        fill='tozeroy'
    ),
    row=2, col=2
)

# Update layout
fig_greeks.update_layout(
    height=800,
    title_text=f"Option Greeks Evolution Analysis - ATM Call (${atm_strike:,} Strike)",
    title_x=0.5,
    showlegend=True
)

# Update axis labels
fig_greeks.update_yaxes(title_text="Spot Price ($)", secondary_y=False, row=1, col=1)
fig_greeks.update_yaxes(title_text="Option Price ($)", secondary_y=True, row=1, col=1)
fig_greeks.update_yaxes(title_text="Delta", row=1, col=2)
fig_greeks.update_yaxes(title_text="Gamma (x10k)", row=2, col=1)
fig_greeks.update_yaxes(title_text="Theta ($)", row=2, col=2)
fig_greeks.update_xaxes(title_text="Moneyness (S/K)", row=3, col=1)
fig_greeks.update_yaxes(title_text="Delta", row=3, col=1)
fig_greeks.update_xaxes(title_text="Days to Expiry", row=3, col=2)
fig_greeks.update_yaxes(title_text="Option Price ($)", row=3, col=2)

fig_greeks.show()

# Print Greeks summary
print("\n🎯 Greeks Analysis Summary:")
print("=" * 40)
print(f"📊 Delta range: {greeks_df['delta'].min():.3f} to {greeks_df['delta'].max():.3f}")
print(f"📊 Max Gamma: {greeks_df['gamma'].max():.6f}")
print(f"📊 Avg daily Theta: ${greeks_df['theta'].mean():.2f}")
print(f"📊 Final option value: ${greeks_df['option_price'].iloc[-1]:.2f}")


🎯 Greeks Analysis Summary:
📊 Delta range: 0.456 to 1.000
📊 Max Gamma: 0.000042
📊 Avg daily Theta: $-178.19
📊 Final option value: $10723.05


## 4. Hedging Activity Analysis
Simulate and visualize delta hedging activity with different thresholds

In [18]:
def simulate_hedging_activity(spot_data, strike, expiry_days, iv, hedge_threshold=0.1):
    """Simulate detailed hedging activity"""
    
    expiry = spot_data['timestamp'].min() + pd.Timedelta(days=expiry_days)
    hedge_position = 0
    last_delta = 0
    
    hedging_data = []
    
    for _, row in spot_data.iterrows():
        spot = row['close']
        timestamp = row['timestamp']
        
        tte = max(0, (expiry - timestamp).total_seconds() / (365 * 24 * 3600))
        
        if tte > 0:
            current_delta = delta(spot, strike, tte, 0.01, iv, 'call')
            current_gamma = gamma(spot, strike, tte, 0.01, iv)
            
            # Check if hedging is needed using delta-band method
            delta_change = abs(current_delta - last_delta)
            needs_hedge = delta_change > hedge_threshold
            
            hedge_trade = 0
            if needs_hedge:
                # Calculate hedge trade size
                target_hedge = -current_delta  # Hedge opposite to delta
                hedge_trade = target_hedge - hedge_position
                hedge_position = target_hedge
                last_delta = current_delta
            
            # Calculate net exposure
            net_delta = current_delta + hedge_position
            
            hedging_data.append({
                'timestamp': timestamp,
                'spot': spot,
                'option_delta': current_delta,
                'hedge_position': hedge_position,
                'net_delta': net_delta,
                'gamma': current_gamma,
                'hedge_trade': hedge_trade,
                'needs_hedge': needs_hedge,
                'threshold': hedge_threshold
            })
    
    return pd.DataFrame(hedging_data)

# Simulate different hedging strategies
print("🔄 Simulating hedging activities...")

hedge_scenarios = [
    {'name': 'Conservative (15%)', 'threshold': 0.15, 'color': 'blue'},
    {'name': 'Moderate (10%)', 'threshold': 0.10, 'color': 'green'},
    {'name': 'Aggressive (5%)', 'threshold': 0.05, 'color': 'orange'},
    {'name': 'Very Aggressive (2.5%)', 'threshold': 0.025, 'color': 'red'}
]

hedge_results = {}
for scenario in hedge_scenarios:
    hedge_results[scenario['name']] = simulate_hedging_activity(
        spot_data=spot_data.iloc[:500],  # First ~20 days
        strike=atm_strike,
        expiry_days=30,
        iv=0.8,
        hedge_threshold=scenario['threshold']
    )
    
    trades = hedge_results[scenario['name']][hedge_results[scenario['name']]['hedge_trade'] != 0]
    print(f"📊 {scenario['name']}: {len(trades)} hedge trades")

print(f"✅ Hedging simulation completed!")

🔄 Simulating hedging activities...
📊 Conservative (15%): 2 hedge trades
📊 Moderate (10%): 7 hedge trades
📊 Aggressive (5%): 20 hedge trades
📊 Very Aggressive (2.5%): 54 hedge trades
✅ Hedging simulation completed!


In [19]:
# Create comprehensive hedging visualization
fig_hedge = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Delta Evolution and Hedge Positions',
        'Net Delta Exposure',
        'Hedge Trade Frequency',
        'Cumulative Hedge Volume'
    ]
)

# Colors for different strategies
colors = ['blue', 'green', 'orange', 'red']

# 1. Delta Evolution and Hedge Positions
base_df = hedge_results['Moderate (10%)']
fig_hedge.add_trace(
    go.Scatter(
        x=base_df['timestamp'],
        y=base_df['option_delta'],
        name='Option Delta',
        line=dict(color='black', width=3)
    ),
    row=1, col=1
)

for i, (name, df) in enumerate(hedge_results.items()):
    fig_hedge.add_trace(
        go.Scatter(
            x=df['timestamp'],
            y=df['hedge_position'],
            name=f'Hedge: {name}',
            line=dict(color=colors[i], width=2, dash='dash')
        ),
        row=1, col=1
    )

# 2. Net Delta Exposure
for i, (name, df) in enumerate(hedge_results.items()):
    fig_hedge.add_trace(
        go.Scatter(
            x=df['timestamp'],
            y=df['net_delta'],
            name=f'Net Δ: {name}',
            line=dict(color=colors[i], width=2),
            fill='tonexty' if i == 0 else None,
            showlegend=False
        ),
        row=1, col=2
    )

fig_hedge.add_hline(y=0, line_dash="solid", line_color="black", row=1, col=2)

# 3. Hedge Trade Frequency (bar chart)
trade_counts = []
strategy_names = []
avg_trade_sizes = []

for name, df in hedge_results.items():
    trades = df[df['hedge_trade'] != 0]
    trade_counts.append(len(trades))
    strategy_names.append(name.split(' (')[0])  # Shorten name
    avg_trade_sizes.append(abs(trades['hedge_trade']).mean() if len(trades) > 0 else 0)

fig_hedge.add_trace(
    go.Bar(
        x=strategy_names,
        y=trade_counts,
        name='Trade Count',
        marker_color=colors,
        showlegend=False
    ),
    row=2, col=1
)

# 4. Cumulative Hedge Volume
for i, (name, df) in enumerate(hedge_results.items()):
    cumulative_volume = abs(df['hedge_trade']).cumsum()
    fig_hedge.add_trace(
        go.Scatter(
            x=df['timestamp'],
            y=cumulative_volume,
            name=f'Vol: {name}',
            line=dict(color=colors[i], width=2),
            showlegend=False
        ),
        row=2, col=2
    )

# Update layout
fig_hedge.update_layout(
    height=800,
    title_text="Delta Hedging Activity Analysis",
    title_x=0.5,
    showlegend=True
)

# Update axis labels
fig_hedge.update_yaxes(title_text="Delta", row=1, col=1)
fig_hedge.update_yaxes(title_text="Net Delta", row=1, col=2)
fig_hedge.update_yaxes(title_text="Number of Trades", row=2, col=1)
fig_hedge.update_yaxes(title_text="Cumulative Volume", row=2, col=2)

fig_hedge.show()

# Print hedging statistics
print("\n🔄 Hedging Activity Summary:")
print("=" * 50)
for i, (name, df) in enumerate(hedge_results.items()):
    trades = df[df['hedge_trade'] != 0]
    max_net_delta = abs(df['net_delta']).max()
    total_volume = abs(df['hedge_trade']).sum()
    
    print(f"\n{name}:")
    print(f"  🔢 Total Trades: {len(trades)}")
    print(f"  📊 Avg Trade Size: {avg_trade_sizes[i]:.4f}")
    print(f"  📈 Max Net Delta: {max_net_delta:.4f}")
    print(f"  📦 Total Volume: {total_volume:.2f}")


🔄 Hedging Activity Summary:

Conservative (15%):
  🔢 Total Trades: 2
  📊 Avg Trade Size: 0.3598
  📈 Max Net Delta: 0.1465
  📦 Total Volume: 0.72

Moderate (10%):
  🔢 Total Trades: 7
  📊 Avg Trade Size: 0.1705
  📈 Max Net Delta: 0.0982
  📦 Total Volume: 1.19

Aggressive (5%):
  🔢 Total Trades: 20
  📊 Avg Trade Size: 0.0873
  📈 Max Net Delta: 0.0498
  📦 Total Volume: 1.75

Very Aggressive (2.5%):
  🔢 Total Trades: 54
  📊 Avg Trade Size: 0.0457
  📈 Max Net Delta: 0.0248
  📦 Total Volume: 2.47


## 5. P&L Attribution Analysis
Detailed breakdown of gamma scalping P&L sources

In [20]:
# Get detailed P&L data from a successful strategy result
if not results.empty:
    # Get the best performing strategy for detailed analysis
    best_strategy = results.nlargest(1, 'total_pnl').iloc[0]
    print(f"🎯 Analyzing detailed P&L for: {best_strategy['strategy']}")
    
    # Create P&L visualization using the actual results data
    fig_pnl = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'P&L Components Distribution',
            'Strategy Performance Comparison',
            'Risk-Return Profile', 
            'Transaction Cost Analysis'
        ],
        specs=[[{'type': 'histogram'}, {'type': 'bar'}],
               [{'type': 'scatter'}, {'type': 'pie'}]]
    )
    
    # 1. P&L Distribution
    fig_pnl.add_trace(
        go.Histogram(
            x=results['total_pnl'],
            name='Total P&L Distribution',
            nbinsx=20,
            marker_color='lightblue',
            opacity=0.7
        ),
        row=1, col=1
    )
    
    # Add mean line
    mean_pnl = results['total_pnl'].mean()
    fig_pnl.add_vline(x=mean_pnl, line_dash="dash", line_color="red", 
                     annotation_text=f"Mean: ${mean_pnl:.0f}", row=1, col=1)
    
    # 2. Strategy Performance Comparison
    strategy_perf = results.groupby('strategy').agg({
        'total_pnl': 'sum',
        'option_pnl': 'sum',
        'hedge_pnl': 'sum'
    }).reset_index()
    
    fig_pnl.add_trace(
        go.Bar(
            x=strategy_perf['strategy'],
            y=strategy_perf['option_pnl'],
            name='Option P&L',
            marker_color='lightcoral'
        ),
        row=1, col=2
    )
    
    fig_pnl.add_trace(
        go.Bar(
            x=strategy_perf['strategy'],
            y=strategy_perf['hedge_pnl'],
            name='Hedge P&L',
            marker_color='lightgreen'
        ),
        row=1, col=2
    )
    
    # 3. Risk-Return Profile
    strategy_stats = results.groupby('strategy').agg({
        'total_pnl': ['mean', 'std']
    }).reset_index()
    strategy_stats.columns = ['strategy', 'avg_pnl', 'pnl_std']
    
    fig_pnl.add_trace(
        go.Scatter(
            x=strategy_stats['pnl_std'],
            y=strategy_stats['avg_pnl'],
            mode='markers+text',
            text=strategy_stats['strategy'],
            textposition='top center',
            marker=dict(size=15, opacity=0.7),
            name='Risk-Return',
            showlegend=False
        ),
        row=2, col=1
    )
    
    # 4. Transaction Cost Breakdown
    total_commission = results['commission_cost'].sum()
    total_slippage = results['slippage_cost'].sum()
    total_pnl_gross = results['total_pnl'].sum() + total_commission + total_slippage
    
    fig_pnl.add_trace(
        go.Pie(
            labels=['Gross P&L', 'Commission', 'Slippage'],
            values=[total_pnl_gross, total_commission, total_slippage],
            name='Cost Breakdown'
        ),
        row=2, col=2
    )
    
    # Update layout
    fig_pnl.update_layout(
        height=800,
        title_text="P&L Attribution and Risk Analysis",
        title_x=0.5,
        showlegend=True
    )
    
    # Update axis labels
    fig_pnl.update_xaxes(title_text="P&L ($)", row=1, col=1)
    fig_pnl.update_yaxes(title_text="Frequency", row=1, col=1)
    fig_pnl.update_yaxes(title_text="P&L ($)", row=1, col=2)
    fig_pnl.update_xaxes(title_text="Risk (Std Dev)", row=2, col=1)
    fig_pnl.update_yaxes(title_text="Return (Mean P&L)", row=2, col=1)
    
    fig_pnl.show()
    
    # Print detailed P&L analysis
    print("\n💰 P&L Attribution Summary:")
    print("=" * 50)
    print(f"📊 Total Strategies Tested: {len(results['strategy'].unique())}")
    print(f"📊 Total Option Contracts: {len(results)}")
    print(f"💰 Total P&L: ${results['total_pnl'].sum():,.2f}")
    print(f"📈 Option P&L: ${results['option_pnl'].sum():,.2f}")
    print(f"🔄 Hedge P&L: ${results['hedge_pnl'].sum():,.2f}")
    print(f"💸 Total Commissions: ${results['commission_cost'].sum():,.2f}")
    print(f"💸 Total Slippage: ${results['slippage_cost'].sum():,.2f}")
    print(f"🎯 Win Rate: {(results['total_pnl'] > 0).mean():.1%}")
    print(f"📊 Best Single Trade: ${results['total_pnl'].max():,.2f}")
    print(f"📊 Worst Single Trade: ${results['total_pnl'].min():,.2f}")

else:
    print("❌ No results available for P&L analysis")

🎯 Analyzing detailed P&L for: Moderate (10%)



💰 P&L Attribution Summary:
📊 Total Strategies Tested: 4
📊 Total Option Contracts: 120
💰 Total P&L: $210,845.83
📈 Option P&L: $198,972.84
🔄 Hedge P&L: $141,156.49
💸 Total Commissions: $43,094.50
💸 Total Slippage: $86,189.00
🎯 Win Rate: 56.7%
📊 Best Single Trade: $10,946.99
📊 Worst Single Trade: $-3,248.17


In [23]:
# Build P&L over time from raw results (cumulative level aggregation)
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Normalize raw_results into a list of (strategy, result) pairs
pairs = []
if isinstance(raw_results_map, dict):
    for strat, res_list in raw_results_map.items():
        for res in (res_list or []):
            pairs.append((strat, res))
elif isinstance(raw_results_map, list):
    for res in raw_results_map:
        strat = getattr(res, 'strategy_name', None) or getattr(res, 'strategy', None) or 'Strategy'
        pairs.append((strat, res))
else:
    pairs = []

# Aggregate P&L histories
rows = []
option_metadata = []
for strat, res in pairs:
    if res is None or getattr(res, 'pnl_history', None) is None:
        continue
    ts_df = res.pnl_history
    if ts_df is None or ts_df.empty or 'timestamp' not in ts_df.columns or 'total_pnl' not in ts_df.columns:
        continue
    ts = ts_df[['timestamp', 'total_pnl']].copy()
    ts['strategy'] = strat
    ts['symbol'] = getattr(res, 'option_symbol', None)
    rows.append(ts)
    
    # Track option lifecycle metadata
    option_metadata.append({
        'strategy': strat,
        'symbol': getattr(res, 'option_symbol', None),
        'strike': getattr(res, 'strike', None),
        'expiry': getattr(res, 'expiry', None),
        'start_time': ts_df['timestamp'].min(),
        'end_time': ts_df['timestamp'].max(),
        'final_pnl': getattr(res, 'total_pnl', None)
    })

if not rows:
    print('No raw pnl histories available to aggregate.')
else:
    ts_raw = pd.concat(rows, ignore_index=True)
    metadata_df = pd.DataFrame(option_metadata)
    
    # Group by strategy and timestamp, taking the mean of already-cumulative total_pnl values
    # (total_pnl in pnl_history is already cumulative for each option)
    ts_agg = (
        ts_raw.groupby(['strategy', 'timestamp'])
              .agg(avg_cumulative_pnl=('total_pnl', 'mean'),  # Average of cumulative P&Ls
                   sum_cumulative_pnl=('total_pnl', 'sum'),   # Sum of cumulative P&Ls
                   active_options=('total_pnl', 'count'))
              .reset_index()
              .sort_values(['strategy', 'timestamp'])
    )

    # Create the visualization with the correct cumulative values
    fig = px.line(ts_agg, x='timestamp', y='avg_cumulative_pnl', color='strategy',
                  title='Average Cumulative P&L Over Time (Per Option)',
                  labels={'avg_cumulative_pnl': 'Avg P&L per Option ($)', 'timestamp': 'Date'})
    fig.update_layout(height=500)
    fig.show()
    
    # Also show total cumulative P&L across all options
    fig_total = px.line(ts_agg, x='timestamp', y='sum_cumulative_pnl', color='strategy',
                        title='Total Cumulative P&L Over Time (All Active Options)',
                        labels={'sum_cumulative_pnl': 'Total P&L ($)', 'timestamp': 'Date'})
    fig_total.update_layout(height=500)
    fig_total.show()

    # Enhanced active options visualization
    fig_active = px.line(ts_agg, x='timestamp', y='active_options', color='strategy',
                         title='Active Options Count Over Time (Options Expire Naturally)',
                         labels={'active_options': 'Number of Active Options', 'timestamp': 'Date'})
    
    # Add vertical lines for expiry dates using shapes instead of add_vline
    if not metadata_df.empty:
        # Get the time range of the plot
        min_time = ts_agg['timestamp'].min()
        max_time = ts_agg['timestamp'].max()
        
        # Get unique expiry dates
        metadata_df['expiry'] = pd.to_datetime(metadata_df['expiry'])
        expiry_dates = metadata_df['expiry'].dropna().unique()
        
        # Filter expiries to only those within the timeframe
        expiry_dates_in_range = [exp for exp in expiry_dates if min_time <= exp <= max_time]
        
        # Add vertical lines as shapes
        shapes = []
        annotations = []
        for i, expiry in enumerate(sorted(expiry_dates_in_range)):
            shapes.append(dict(
                type='line',
                x0=expiry, x1=expiry,
                y0=0, y1=1,
                yref='paper',
                line=dict(color='gray', width=1, dash='dash'),
                opacity=0.3
            ))
            # Add annotation for first few expiries only
            if i < 3:
                annotations.append(dict(
                    x=expiry,
                    y=1,
                    yref='paper',
                    text='Expiry',
                    showarrow=False,
                    textangle=-90,
                    xanchor='left',
                    yanchor='bottom',
                    font=dict(size=10, color='gray')
                ))
        
        fig_active.update_layout(shapes=shapes, annotations=annotations)
    
    fig_active.update_layout(height=400, showlegend=True)
    fig_active.show()
    
    # Print lifecycle analysis
    print("\n📊 Option Lifecycle Analysis:")
    print("=" * 50)
    print(f"Total options traded: {len(metadata_df['symbol'].unique())}")
    print(f"Options expiring during simulation: Expected behavior")
    print("\nNote: The decreasing option count is EXPECTED as options naturally")
    print("expire throughout the simulation period. In production, you would")
    print("continuously roll to new options as old ones approach expiry.")
    
    # Show expiry schedule (only those within the simulation timeframe)
    if not metadata_df.empty:
        min_time = ts_agg['timestamp'].min()
        max_time = ts_agg['timestamp'].max()
        
        expiry_schedule = metadata_df.groupby('expiry').size().reset_index(name='count')
        expiry_schedule = expiry_schedule.sort_values('expiry')
        # Filter to show only expiries within the simulation timeframe
        expiry_schedule = expiry_schedule[(expiry_schedule['expiry'] >= min_time) & 
                                          (expiry_schedule['expiry'] <= max_time)]
        
        print("\n📅 Option Expiry Schedule (within simulation period):")
        for _, row in expiry_schedule.head(10).iterrows():
            if pd.notna(row['expiry']):
                print(f"  {pd.Timestamp(row['expiry']).date()}: {row['count']} options expire")
    
    # Print final P&L values
    print("\n📊 Final P&L Values (at last timestamp with data):")
    print("=" * 50)
    final_values = ts_agg.groupby('strategy').last()
    for strategy in final_values.index:
        row = final_values.loc[strategy]
        print(f"{strategy}:")
        print(f"  Average P&L per option: ${row['avg_cumulative_pnl']:,.2f}")
        print(f"  Total P&L: ${row['sum_cumulative_pnl']:,.2f}")
        print(f"  Active options: {int(row['active_options'])}")


📊 Option Lifecycle Analysis:
Total options traded: 30
Options expiring during simulation: Expected behavior

Note: The decreasing option count is EXPECTED as options naturally
expire throughout the simulation period. In production, you would
continuously roll to new options as old ones approach expiry.

📅 Option Expiry Schedule (within simulation period):
  2025-01-24: 28 options expire
  2025-01-31: 12 options expire
  2025-02-07: 36 options expire
  2025-02-14: 8 options expire
  2025-02-21: 16 options expire

📊 Final P&L Values (at last timestamp with data):
Aggressive (5%):
  Average P&L per option: $249.08
  Total P&L: $249.08
  Active options: 1
Conservative (15%):
  Average P&L per option: $249.08
  Total P&L: $249.08
  Active options: 1
Moderate (10%):
  Average P&L per option: $249.08
  Total P&L: $249.08
  Active options: 1
Very Aggressive (2.5%):
  Average P&L per option: $265.73
  Total P&L: $265.73
  Active options: 1
